# LangChain

In [1]:
! pip list | grep langchain

langchain                     0.0.274


In [3]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo")
response = chat([HumanMessage(content="Translate this sentence from English to French: I love programming.")])

In [8]:
response.content

"J'adore programmer."

In [11]:
response.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'messages', 'AIMessage'],
 'kwargs': {'content': "J'adore programmer.", 'additional_kwargs': {}}}

# Memory

Memory allows the AI to remember the context of human interactions. This memory is preserved by creating summaries.

In [15]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("Hi! Nice to meet you, I'm Shem and I am a programer!")
memory.chat_memory.add_ai_message("Oh! Nice to meet you too, I'm Audrey and I also a programer!")

In [16]:
memory.load_memory_variables({}) # when you use Buffer Memory, memory will completly saved.

{'history': "Human: Hi! Nice to meet you, I'm Shem and I am a programer!\nAI: Oh! Nice to meet you too, I'm Audrey and I also a programer!"}

In [20]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.chat_memory.add_user_message("Hi! Nice to meet you, I'm Shem and I am a programer!")
memory.chat_memory.add_ai_message("Oh! Nice to meet you too, I'm Audrey and I also a programer!")
memory.chat_memory.add_user_message("Hi! Nice to meet you, I'm Jack and I am a house keeper!")
memory.chat_memory.add_ai_message("Oh! Nice to meet you too, I'm Black and I also a house keeper!")
memory.load_memory_variables({}) # When you use a window buffer, the memory is saved based on the 'k' steps you've set.

{'history': "Human: Hi! Nice to meet you, I'm Jack and I am a house keeper!\nAI: Oh! Nice to meet you too, I'm Black and I also a house keeper!"}

In [32]:
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryMemory

llm = OpenAI(temperature=0)
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)
memory.save_context(inputs={"User":"Hi! Nice to meet you, I'm Shem and I am a programer!"}, outputs={"AI":"Oh! Nice to meet you too, I'm Audrey and I also a programer!"})

In [33]:
memory.load_memory_variables({})

{'chat_history': [SystemMessage(content='\nThe human introduces themselves as Shem, a programmer, and the AI introduces themselves as Audrey, also a programmer.', additional_kwargs={})]}

## Impletment chat with memory

Using the SummaryMemory we just created, we can ask the AI for the name we added to the memory.

In [34]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain

# LLM
llm = ChatOpenAI()

# Prompt 
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "Do you remember whats my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
System: 
The human introduces themselves as Shem, a programmer, and the AI introduces themselves as Audrey, also a programmer.
Human: Do you remember whats my name?

> Finished chain.


{'question': 'Do you remember whats my name?',
 'chat_history': [SystemMessage(content='\nThe human introduces themselves as Shem, a programmer, and the AI introduces themselves as Audrey, also a programmer.', additional_kwargs={})],
 'text': "Yes, your name is Shem. It's nice to meet you, Shem! How can I assist you today?"}